In [1]:
import os
import numpy as np

from dotenv import load_dotenv


load_dotenv("../.env")

True

In [2]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS


In [3]:
# Load and split documents
loader_pdf = PyPDFLoader("/Users/eugenekillevsky/PycharmProjects/LLM-Interviewer/data/RAG/Deep Learning-Ian Goodfellow.pdf")

documents = loader_pdf.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(documents)

len(chunks)

4179

In [4]:
len(documents)

800

In [5]:
documents[502]

Document(metadata={'source': '/Users/eugenekillevsky/PycharmProjects/LLM-Interviewer/data/RAG/Deep Learning-Ian Goodfellow.pdf', 'page': 502}, page_content='a, b and c. In order to even compute such conditional probabilities one needs\nto sum over the values of the variables c, as well as compute a normalization\nconstant which sums over the values of a and c.\n• Intractable normalization constants (the partition function): the partition\nfunction is discussed mostly in chapter . Normalizing constants of proba-18\nbility functions come up in inference (above) as well as in learning. Many\nprobabilistic models involve such a normalizing constant. Unfortunately,\nlearning such a model often requires computing the gradient of the loga-\nrithm of the partition function with respect to the model parameters. That\ncomputation is generally as intractable as computing the partition function\nitself. Monte Carlo Markov chain (MCMC) methods (chapter ) are of-17\nten used to deal with the partiti

In [12]:
from langchain_mistralai.embeddings import MistralAIEmbeddings
from langchain_openai import OpenAIEmbeddings
from tokenizers import Tokenizer


embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
    api_key=os.getenv("OPENAI_API_KEY"),
    chunk_size=1,
    dimensions=1024,
    show_progress_bar=True
)

In [14]:
from langchain_chroma import Chroma

# vector_store = FAISS.from_documents(chunks, embeddings)
vector_store = Chroma(
    collection_name="ian_goodfellow_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db"
)
vector_store.add_documents(chunks)

  0%|          | 0/4179 [00:00<?, ?it/s]

['739efb41-23c5-41bf-9481-d905c526a806',
 '9113c75d-88bb-4a59-b092-5a624c0b5b67',
 '6d57df0c-6fc9-4823-a052-5bf7da01ccab',
 '9fc69f7d-d651-43d6-b14a-b4a36d86d56c',
 '20f52b06-dfe1-4fc0-a0d0-d2ff95210beb',
 'ff75276b-ca0b-4c51-b75b-e179eeb011ae',
 '107d3224-946f-4ffe-9186-e95bbb32a167',
 'c3bb8151-d315-412a-beeb-696dfdc75985',
 '0ea29d2f-f799-46ab-8304-5ed42d112c4f',
 'caa4dda9-1239-4625-8dbc-6dc0276ff117',
 'b68d3096-e318-4f1e-aa9e-c422bfdf9e5f',
 'e00969dc-092b-4a02-b153-6934a000086b',
 '7c4d1a58-6b68-4661-938d-b674fa981604',
 '12087d41-4ecf-4cab-abea-ca6ba6f09ef6',
 'b2eef1c1-2c10-498e-8fd6-b2f92df0d616',
 'e05ca89c-3ea7-4def-b082-cc527f9ab3cd',
 'f0a57d8c-c12c-4654-b2b7-5abe553c5d15',
 '6e0123b0-763f-4773-9c3b-7ee4d0aa908d',
 'ae36a174-7d0f-4495-bddc-86ab652eda05',
 'd33398f6-2363-46b3-98b6-ec8c4c692370',
 'eaed8e8b-4995-4af5-8cdf-5e6bc4b5321d',
 '04aa4475-d0ac-4d83-8bad-0f88bc36f0cf',
 'e7c3645e-9362-4c87-a94d-1e87082f7097',
 'a745d6ca-2810-4810-b4ea-ea1ff6ab66fa',
 'f9ee70fc-c276-

In [19]:
results = vector_store.similarity_search_with_score(
    "Deep Learning", k=3
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

  0%|          | 0/1 [00:00<?, ?it/s]

* [SIM=0.734127] • Deep learninghas solvedincreasingly complicated applicationswith increasing
accuracy over time.
11 [{'page': 25, 'source': '/Users/eugenekillevsky/PycharmProjects/LLM-Interviewer/data/RAG/Deep Learning-Ian Goodfellow.pdf'}]
* [SIM=0.750149] nologies that are already used heavily in industry.
Modern deep learning provides a very powerful framework for supervised
learning. By adding more layers and more units within a layer, a deep network can
represent functions of increasing complexity. Most tasks that consist of mapping an
input vector to an output vector, and that are easy for a person to do rapidly, can
be accomplished via deep learning, given suﬃciently large models and suﬃciently [{'page': 181, 'source': '/Users/eugenekillevsky/PycharmProjects/LLM-Interviewer/data/RAG/Deep Learning-Ian Goodfellow.pdf'}]
* [SIM=0.776836] initially interesting because it was able to generalize to new examples better
than competing algorithms when trained on medium-sized datasets w

In [22]:
from langchain_openai import OpenAI

llm = OpenAI(
    model="gpt-4o-mini"
)

In [28]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_mistralai.chat_models import ChatMistralAI


# Define LLM
model = ChatMistralAI(mistral_api_key=os.getenv("MISTRAL_KEY"))

# Create retriever
retriever = vector_store.as_retriever()

# Prompt
prompt = ChatPromptTemplate.from_template("""Generate the question for the interviewee based on the selected topic of the interview:
{input}
And provided context (it's text of the book):

<context>
{context}
</context>
""")

# Create a retrieval chain to answer questions
document_chain = create_stuff_documents_chain(model, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)
response = retrieval_chain.invoke({"input": "Natural Language Processing"})
print(response["answer"])

  0%|          | 0/1 [00:00<?, ?it/s]

Based on the provided context, one possible question for an interviewee could be:

"Can you explain what natural language processing (NLP) is, and describe some of the domain-specific strategies that can be used to build an efficient NLP model that can achieve excellent performance and scale well to large applications? Additionally, can you discuss the role of neural language models (NLMs) in overcoming the curse of dimensionality problem for modeling natural language sequences?"


In [35]:
ml_handbook_path = "/Users/eugenekillevsky/PycharmProjects/LLM-Interviewer/data/RAG/ml-handbook-master/chapters"
os.listdir(os.path.join(ml_handbook_path, "ensembles"))

['images', '.gitkeep', 'intro.md']

In [37]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_core.documents import Document


loader = UnstructuredMarkdownLoader(os.path.join(ml_handbook_path, "ensembles", "intro.md"))

data = loader.load()
data_content = data[0].page_content
print(len(readme_content))

15443


In [41]:
split_string_list = [data_content[x:x+500] for x in range(0, len(data_content), 500)]
len(split_string_list)

31

In [45]:
for str_ in split_string_list[2:]:
    print(len(str_.split(' ')))
    print(str_)
    break

77
 \eps} [y(x, \eps) - a(x, X)]^2, $$

где

$X$ -- обучающая выборка

$x$ -- точка из тестового множества

$y = f(x) + \eps$ -- целевая зависимость, которую мы можем измерить с точностью до случайного шума $\eps$

$a(x, X)$ -- значение алгоритма, обученного на выборке $X$, в точке $x$

$\Exp_x$ -- среднее по всем тестовым точкам и $\Exp_{X, \eps}$ -- среднее по всем обучающим выборкам $X$ и случайному шуму $\eps$

Для $Q(a)$ существует разложение на три компоненты -- шум, смещение и разброс. Это р


## Статистика по Гудфеллоу

In [71]:
from langchain.document_loaders import PyPDFLoader

# Load and split documents
loader_pdf = PyPDFLoader("/Users/eugenekillevsky/PycharmProjects/LLM-Interviewer/data/RAG/Deep Learning-Ian Goodfellow.pdf")

documents = loader_pdf.load()

In [72]:
len(documents)

800

In [77]:
from tqdm import tqdm

total_symbols = 0
total_words = 0

for doc in tqdm(documents):
    doc_data = doc.page_content
    total_symbols += len(doc_data)
    total_words += len(doc_data.split(' '))

print(f"\nTotal: {total_symbols} symbols, {total_words} words")

100%|██████████| 800/800 [00:00<00:00, 41878.13it/s]


Total: 1768399 symbols, 263367 words


## Статистика по Хэндбуку

In [73]:
print("| Название главы | Количество символов | Количество слов |")

total_symbols = 0
total_words = 0
for chapter_name in os.listdir(ml_handbook_path):
    chapter_md_path = os.path.join(ml_handbook_path, chapter_name)
    valid_mds = [
        file_path
        for file_path in os.listdir(chapter_md_path)
        if file_path.endswith(".md")
    ]

    chapter_num_symbols = 0
    chapter_num_words = 0
    for valid_md_name in valid_mds:
        md_loader = UnstructuredMarkdownLoader(os.path.join(chapter_md_path, valid_md_name))
        md_data = md_loader.load()
        md_data_content = md_data[0].page_content

        chapter_num_symbols += len(md_data_content)
        chapter_num_words += len(md_data_content.split(' '))

    total_symbols += chapter_num_symbols
    total_words += chapter_num_words

    print("|", chapter_name, "|", chapter_num_symbols, "|", chapter_num_words, "|")

print(f"\nTotal: {total_symbols} symbols, {total_words} words")

| Название главы | Количество символов | Количество слов |
| prob_genclass | 16305 | 1972 |
| ensembles | 15443 | 2075 |
| cross_validation | 21843 | 3057 |
| neural_nets | 72885 | 8776 |
| matrix_diff | 11821 | 1386 |
| model_evaluation | 42767 | 5644 |
| intro | 33733 | 4637 |
| prob_maxent | 13497 | 1494 |
| prob_bayes | 35694 | 4135 |
| optimization | 89547 | 11260 |
| decision_tree | 40123 | 5294 |
| prob_calibration | 12381 | 1499 |
| clustering | 33116 | 4220 |
| prob_intro | 17075 | 2122 |
| grad_boost | 20575 | 2574 |
| hyperparameters_tuning | 32705 | 4214 |
| prob_glm | 10117 | 1255 |
| ml_theory | 13547 | 1799 |
| linear_models | 74505 | 9865 |
| metric_based | 33264 | 4217 |

Total: 640943 symbols, 81495 words


## RAG по Хэндбуку

### Строим

In [54]:
from tqdm import tqdm

chapters_documents = []

for chapter_name in tqdm(os.listdir(ml_handbook_path)):
    chapter_md_path = os.path.join(ml_handbook_path, chapter_name)
    valid_mds = [
        file_path
        for file_path in os.listdir(chapter_md_path)
        if file_path.endswith(".md")
    ]

    for valid_md_name in valid_mds:
        valid_md_path = os.path.join(chapter_md_path, valid_md_name)
        md_loader = UnstructuredMarkdownLoader(valid_md_path)
        md_data = md_loader.load()
        # md_data_content = md_data[0].page_content
        # print(chapter_name, valid_md_name, len(md_data_content))
        chapters_documents.append(md_data[0])


100%|██████████| 20/20 [00:01<00:00, 13.17it/s]


In [57]:
vector_store_yandex = Chroma(
    collection_name="ml_yandex_handbook",
    embedding_function=embeddings,
    persist_directory="./chroma_yandex_db"
)
yandex_ids = vector_store_yandex.add_documents(chapters_documents)
yandex_ids[0]

  0%|          | 0/45 [00:00<?, ?it/s]

'd9184a0c-917d-4364-b08b-70ad4c5b7a69'

### Тестируем

In [6]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings


embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
    api_key=os.getenv("OPENAI_API_KEY"),
    chunk_size=1,
    dimensions=1024,
    show_progress_bar=True
)

vectore_store_yandex = Chroma(
    collection_name="ml_yandex_handbook",
    embedding_function=embeddings,
    persist_directory="./chroma_yandex_db"
)

vectore_store_yandex

In [16]:
results = vectore_store_yandex.similarity_search_by_vector(
    embedding=embeddings.embed_query("Глубокое обучение"), k=1
)
for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")

  0%|          | 0/1 [00:00<?, ?it/s]

* title: Нейронные сети author: radoslav_neichev, filipp_sinicin, stanislav_fedotov cover_file_path: ./src/intro_cover.png

Этот список будет заменен оглавлением, за вычетом заголовка "Contents", к которому добавлен класс no_toc. {:toc}

В этой главе вы познакомитесь с нейронными сетями – семейством моделей, которое начиная с 2012-го постепенно добивается превосходства во всё новых и новых приложениях, во многих став де-факто стандартом. Они были придуманы ещё в 70-х, но техническая возможность и понимание того, как обучать нейросети большого размера, появились лишь примерно к 2011 году, и это дало мощный толчок к их развитию. Совокупность нейросетевых подходов и сама наука о нейросетях носит название глубинного обучения или deep learning. Во многом глубинное обучение основано на двух идеях. Во-первых, это стремление к переходу от построения сложных пайплайнов, каждая компонента которых тренируется сама по себе решать кусочек задачи, к end-to-end обучению всей системы, как одного целог

In [17]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain

from langchain_openai import ChatOpenAI


model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

# Create retriever
retriever_yandex = vectore_store_yandex.as_retriever()

# Prompt
prompt = ChatPromptTemplate.from_template("""Generate the questions for the interviewee based on the selected topic of the interview:
{input}
And provided context (it's text of the book):

<context>
{context}
</context>

Генерировать надо на русском. Нужно сгенерировать как вопрос, так и ответ на этот вопрос, используя лишь приведенную информацию. В формате

Q: ...
A: ...

Всего нужно сгенерировать 20 вопросов.
""")

# Create a retrieval chain to answer questions
document_chain = create_stuff_documents_chain(model, prompt)
retrieval_chain = create_retrieval_chain(retriever_yandex, document_chain)
response = retrieval_chain.invoke({"input": "Глубокое обучение"})
print(response["answer"])

  0%|          | 0/1 [00:00<?, ?it/s]

Q: Что такое глубинное обучение и как оно связано с нейронными сетями?  
A: Глубинное обучение — это область машинного обучения, основанная на нейронных сетях, которая позволяет обучать модели, способные решать сложные задачи, используя большие объемы данных. Нейронные сети, начиная с 2012 года, стали стандартом в многих приложениях благодаря своей гибкости и способности к автоматическому обучению представлений объектов.

Q: Какие две ключевые идеи лежат в основе глубинного обучения?  
A: Первая идея — это переход к end-to-end обучению всей системы, что позволяет обучать все слои нейросети одновременно. Вторая идея — это автоматизация процесса отбора признаков, что позволяет извлекать информативные представления объектов на основе самих данных, часто неразмеченных.

Q: Почему современные нейронные сети сложнее своих предшественников?  
A: Современные нейронные сети сложнее из-за роста производительности компьютеров, увеличения объемов доступных данных и потребностей индустрии, что прив

In [60]:
from langchain_openai import ChatOpenAI


model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    # max_tokens=None,
    # timeout=None,
    # max_retries=2,
    # api_key="...",  # if you prefer to pass api key in directly instaed of using env vars
    # base_url="...",
    # organization="...",
    # other params...
)

# Create retriever
retriever_yandex = vector_store_yandex.as_retriever()

# Prompt
prompt = ChatPromptTemplate.from_template("""Generate the questions for the interviewee based on the selected topic of the interview:
{input}
And provided context (it's text of the book):

<context>
{context}
</context>

Генерировать надо на русском. Нужно сгенерировать как вопрос, так и ответ на этот вопрос, используя лишь приведенную информацию. В формате

Q: ...
A: ...

Всего нужно сгенерировать 20 вопросов.
""")

# Create a retrieval chain to answer questions
document_chain = create_stuff_documents_chain(model, prompt)
retrieval_chain = create_retrieval_chain(retriever_yandex, document_chain)
response = retrieval_chain.invoke({"input": "Классическое машинное обучение"})
print(response["answer"])

  0%|          | 0/1 [00:00<?, ?it/s]

Q: Что такое машинное обучение и какие задачи оно решает?
A: Машинное обучение — это наука, изучающая алгоритмы, которые автоматически улучшаются благодаря опыту. Оно решает задачи, которые трудно или невозможно программировать, такие как перевод текста, диагностика заболеваний, оценка цен и распознавание изображений.

Q: Какова цель построения модели в машинном обучении?
A: Цель построения модели заключается в том, чтобы создать функцию, которая отображает объекты (примеры) в предсказания (таргеты) таким образом, чтобы предсказания были достаточно хорошими, что обычно измеряется с помощью метрик качества.

Q: Какие метрики качества предсказаний могут использоваться в задачах диагностики заболеваний?
A: В задачах диагностики могут использоваться метрики, такие как доля правильно поставленных диагнозов (accuracy) или доля больных, которым удалось поставить правильный диагноз (precision).

Q: Что такое обучающая выборка и из чего она состоит?
A: Обучающая выборка — это набор примеров, со

In [14]:
OpenAIEmbeddings?

Init signature:
OpenAIEmbeddings(
    *,
    client: Any = None,
    async_client: Any = None,
    model: str = 'text-embedding-ada-002',
    dimensions: Optional[int] = None,
    deployment: Optional[str] = 'text-embedding-ada-002',
    api_version: Optional[str] = None,
    base_url: Optional[str] = None,
    openai_api_type: Optional[str] = None,
    openai_proxy: Optional[str] = None,
    embedding_ctx_length: int = 8191,
    api_key: Optional[pydantic.types.SecretStr] = None,
    organization: Optional[str] = None,
    allowed_special: Union[Literal['all'], Set[str], NoneType] = None,
    disallowed_special: Union[Literal['all'], Set[str], Sequence[str], NoneType] = None,
    chunk_size: int = 1000,
    max_retries: int = 2,
    timeout: Union[float, Tuple[float, float], Any, NoneType] = None,
    headers: Any = None,
    tiktoken_enabled: bool = True,
    tiktoken_model_name: Optional[str] = None,
    show_progress_bar: bool = False,
    model_kwargs: Dict[str, Any] = None,
    s